---
layout: default
title: "Elev PTse Downgrade"
parent: "Point Sources"
grand_parent: "Emission Processing"
nav_order: 6
date: 2021-12-08 15:49:27
last_modified_date:   2021-12-08 15:49:19
---

# CAMx高空排放檔之網格化
{: .no_toc }

<details open markdown="block">
  <summary>
    Table of contents
  </summary>
  {: .text-delta }
- TOC
{:toc}
</details>
---

## 背景
高空點源排放檔案沒有適用的顯示軟體。須轉成其他格式，此處以`d04`範圍地面排放量檔案格式為目標，該格式可以在[VERDI](https://github.com/CEMPD/VERDI/blob/master/doc/User_Manual/VERDI_ch01.md)或[MeteoInfo](http://meteothink.org/)中開啟。

## 程式說明

### 程式執行
- [pt2em_d04.py](https://raw.githubusercontent.com/sinotec2/Focus-on-Air-Quality/main/EmisProc/ptse/pt2em_d04.py)只需要一個引數，就是CAMx點源排放量檔案。
- 程式會以`template_d4.nc`為模版，將點源排放量予以網格化填入模版相對應位置。
- 時間標籤則與輸入檔案一致。



In [1]:
pt2em_d04.py fortBE.413_teds10.ptsE01.nc


- 執行結果檔案會再輸入檔名稱後加上`_d04.nc`，以標示其網格系統特性。

### 程式分段說明
- 調用模組



In [2]:
#kuang@node03 /nas1/TEDS/teds11/ptse
#$ cat -n pt2em_d04.py
import netCDF4
import numpy as np
import datetime
import os, sys, subprocess
from pandas import *


- 重要相依性
  - 取得`ncks`、`ncatted`等程式之位置
  - 取得引數(高空點源檔案名稱)



In [3]:
ncks=subprocess.check_output('which ncks',shell=True).decode('utf8').strip('\n')
ncatted=subprocess.check_output('which ncatted',shell=True).decode('utf8').strip('\n')
MM='fortBE.413_teds11.ptsE02.nc'#sys.argv[1]
fname=MM


- 讀取高空排放量檔案(內設為CAMx 7版本)
  - 變數讀取是最花時間的步驟



In [5]:
#store the point source matrix
nct = netCDF4.Dataset(fname,'r')
Vt=[list(filter(lambda x:nct.variables[x].ndim==j, [i for i in nct.variables])) for j in [1,2,3,4]]
ntt,nvt,dt=nct.variables[Vt[2][0]].shape
try:
  nopts=nct.NOPTS
except:
  nopts=nct.dimensions['COL'].size
TFLAG=nct.variables['TFLAG'][:,0,:]
ETFLAG=nct.variables['ETFLAG'][:,0,:]
SDATE=nct.SDATE
STIME=nct.STIME
Vt1=[i for i in Vt[1] if i not in ['CP_NO','plumerise']]
var=np.zeros(shape=(len(Vt1),ntt,nopts))
for v in Vt1:
  iv=Vt1.index(v)
  var[iv,:,:]=nct.variables[v][:,:]


In [11]:
mx=np.max(var)
idx=np.where(var==mx)
print(Vt1[idx[0][0]],mx)
print([(v,np.array(nct.variables[v][:])[idx[2][0]]) for v in Vt[0]])

FPRM 2815001.5
[('pigflag', 0), ('saoverride', -2147483647), ('stkdiam', 7.7878523), ('stkheight', 120.0), ('stkspeed', 57960.0), ('stktemp', 389.0), ('xcoord', -31599.686), ('ycoord', 94159.58)]


- 開啟模版，並讀取網格系統之設定內容，用以計算網格位置標籤。



In [17]:
fname='/nas1/TEDS/teds11/ptse/'+MM+'_d04.nc'
#os.system('cp template_d4.nc '+fname)
nc = netCDF4.Dataset(fname,'r+')
V=[list(filter(lambda x:nc.variables[x].ndim==j, [i for i in nc.variables])) for j in [1,2,3,4]]
nt,nlay,nrow,ncol=nc.variables[V[3][0]].shape
#determination of camx version and prepare IX/IY
ver=7
if 'XSTK' in Vt[0]:ver=6
X={6:'XSTK',7:'xcoord'}
Y={6:'YSTK',7:'ycoord'}
#store the coordinate system param. for calibration
for c in ['X','Y']:
  for d in ['ORIG','CELL']:
    exec(c+d+'=nc.'+c+d)
print(XORIG,YORIG,XCELL,YCELL)

-124500.0 -205500.0 3000.0 3000.0


- 計算每根煙道的網格位置標籤(`IX`, `IY`)備用



In [18]:
IX=np.array([(i-nc.XORIG)/nc.XCELL for i in nct.variables[X[ver]][:nopts]],dtype=int)
IY=np.array([(i-nc.YORIG)/nc.XCELL for i in nct.variables[Y[ver]][:nopts]],dtype=int)
nct.close()
nc.close()


- 篩選非為0的內容來輸出
  - 如果確實有部分變數沒有內容，則從模版中予以去除，以減少檔案容量



In [20]:
#variable sets interception and with values
sint=[v for v in set(Vt1)&set(V[3]) if np.sum(var[Vt1.index(v),:,:])!=0.]
print(len(sint), len(V[3]))
if len(sint)!=len(V[3]):
  s=''.join([c+',' for c in set(V[3])-set(sint)])
  ftmp=fname+'tmp'
  res=os.system(ncks+' -O -x -v'+s.strip(',')+' '+fname+' '+ftmp)
  if res!=0: sys.exit(ncks+' -x var fail')
  ns=str(len(sint)-1)
  res=os.system(ncks+' -O -d VAR,0,'+ns+' '+ftmp+' '+fname)
  if res!=0: sys.exit(ncks+' -d VAR fail')
#template is OK


23 35


SystemExit: /usr/bin/ncks -x var fail

- 執行`pandas.pivot_table`，以利用其平行處理功能。
  - 須將矩陣轉為資料表



In [22]:
#pivoting
ntm,NREC=ntt,nopts
sdt,ix,iy=(np.zeros(shape=(ntm*NREC),dtype=int) for i in range(3))
idatetime=np.array([i for i in range(ntt)],dtype=int)
for t in range(ntm):
    t1,t2=t*NREC,(t+1)*NREC
    ix[t1:t2]=IX
    iy[t1:t2]=IY
for t in range(ntm):
    t1,t2=t*NREC,(t+1)*NREC
    sdt[t1:t2]=idatetime[t]
dfT=DataFrame({'YJH':sdt,'IX':ix,'IY':iy})
for v in sint:
  iv=Vt1.index(v)
  dfT[v]=var[iv,:,:].flatten()
pv=pivot_table(dfT,index=['YJH','IX','IY'],values=sint,aggfunc=sum).reset_index()
pv.head()

,YJH,IX,IY,ALD2,ALDX,BENZ,CCRS,CPRM,ETH,ETHA,...,MEOH,NO,NO2,OLE,PAR,POA,SO2,TERP,TOL,XYL
0,0,-47,98,0.0,0.0,0.0,0.000000,21.130952,0.0,0.0,...,0.0,7176.601254,797.400281,0.0,0.0,0.0,418.430357,0.0,0.0,0.0
1,0,-46,97,0.0,0.0,0.0,4.413224,0.466853,0.0,0.0,...,0.0,21.535897,2.392878,0.0,0.0,0.0,0.382966,0.0,0.0,0.0
2,0,-46,98,0.0,0.0,0.0,8.593750,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0,-45,98,0.0,0.0,0.0,1.563959,1.218991,0.0,0.0,...,0.0,42.053254,4.672585,0.0,0.0,0.0,0.618513,0.0,0.0,0.0
4,0,-43,98,0.0,0.0,0.0,0.000000,2.358059,0.0,0.0,...,0.0,820.090761,91.121212,0.0,0.0,0.0,1.978165,0.0,0.0,0.0


- 再將`pivot_table`結果轉成矩陣輸出



In [23]:
pv.IX=[int(i) for i in pv.IX]
pv.IY=[int(i) for i in pv.IY]
pv.YJH=[int(i) for i in pv.YJH]
boo=(pv.IX>=0) & (pv.IY>=0) & (pv.IX<ncol) & (pv.IY<nrow)
pv=pv.loc[boo].reset_index(drop=True)
imn,jmn=min(pv.IX),min(pv.IY)
imx,jmx=max(max(pv.IX)+abs(imn)*2+1,ncol), max(max(pv.IY)+abs(jmn)*2+1,nrow)
if imn<0 and imx+imn<ncol:sys.exit('negative indexing error in i')
if jmn<0 and jmx+jmn<nrow:sys.exit('negative indexing error in j')
idx=pv.index
idt=np.array(pv.loc[idx,'YJH'])
iy=np.array(pv.loc[idx,'IY'])
ix=np.array(pv.loc[idx,'IX'])
#reopen nc files and write time flags, and lengthen the span of time
nc = netCDF4.Dataset(fname,'r+')
for t in range(ntt):
  for i in range(2):
    nc.variables['TFLAG'][t,:,i]=TFLAG[t,i]
    nc.variables['ETFLAG'][t,:,i]=ETFLAG[t,i]
nc.SDATE=SDATE
nc.STIME=STIME
#blanking all variables
for c in sint:
  nc.variables[c][:]=0.
  z=np.zeros(shape=(ntm,jmx,imx))
  ss=np.array(pv.loc[idx,c])
  #Note that negative indices are not bothersome and are only at the end of the axis.
  z[idt,iy,ix]=ss
#also mapping whole matrix, NOT by parts
  nc.variables[c][:,0,:,:]=z[:,:nrow,:ncol]
nc.close()


- 座標微調
  - 這一段是早期使用twd97座標系統套用VERDI(內政部縣市`shape`檔)時的誤差，改用經緯度後已無需要執行。



In [24]:
#using CSC and XieHePP to calibrate the Map
xiheIXY_Verdi=(67,126) #fallen in the sea
xiheIXY_Target=(66,124)#calibrate with County border and seashore line
CSCIXY_Verdi=(20,30) #fallen in the KSHarbor
CSCIXY_Target=(21,31)
rateXY=np.array([(xiheIXY_Target[i]-CSCIXY_Target[i])/(xiheIXY_Verdi[i]-CSCIXY_Verdi[i]) for i in range(2)])
dxy_new=rateXY*np.array([XCELL,YCELL])
oxy_new=(1-rateXY)*dxy_new*np.array([ncol,nrow])/2.+np.array([XORIG,YORIG])
cmd1=' -a XCELL,global,o,f,'+str(dxy_new[0])
cmd2=' -a YCELL,global,o,f,'+str(dxy_new[1])
cmd3=' -a XORIG,global,o,f,'+str(oxy_new[0])
cmd4=' -a YORIG,global,o,f,'+str(oxy_new[1])
#ncatted -a XCELL,global,o,f,2872.340425531915 -a YCELL,global,o,f,2906.25 -a XORIG,global,o,f,-119074.46808510639 -a YORIG,global,o,f,-199078.125 fortBE.413_teds10.ptsE01.nc_d04.nc
#res=os.system(ncatted+cmd1+cmd2+cmd3+cmd4+' '+fname)
#if res!=0:sys.exit('fail ncatted')
sys.exit('fine!')


SystemExit: fine!

/opt/miniconda3/envs/geocat/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)



## 結果檢視
- [TEDS11高空2月排放檔案之網格分布](https://raw.githubusercontent.com/sinotec2/Focus-on-Air-Quality/main/assets/images/pt2em_d04Demo.PNG)
![](https://github.com/sinotec2/Focus-on-Air-Quality/raw/main/assets/images/pt2em_d04Demo.PNG)
- [台中電廠之NO排放之時間變化](https://github.com/sinotec2/Focus-on-Air-Quality/raw/main/assets/images/pt2em_d04DemoTimVar.PNG)
  - 使用VERDI **Fast Tile Plot**、局部放大、再選取**Plot**、下拉**Time Series of Probed Cell(s)**。
![](https://github.com/sinotec2/Focus-on-Air-Quality/raw/main/assets/images/pt2em_d04DemoTimVar.PNG)
## 檔案下載
- `python`程式：[pt2em_d04.py](https://raw.githubusercontent.com/sinotec2/Focus-on-Air-Quality/main/EmisProc/ptse/pt2em_d04.py)。


## Reference
- lizadams, **Visualization Environment for Rich Data Interpretation (VERDI): User’s Manual**, [github](https://github.com/CEMPD/VERDI/blob/master/doc/User_Manual/VERDI_ch01.md), August 03, 2021
- Yaqiang Wang, **MeteoInfo Introduction**, [meteothink](http://meteothink.org/), 2021,10,16